In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import seaborn as sns
import holidays
import requests
from bs4 import BeautifulSoup 

In [2]:
#Load in original data
data = pd.read_csv('Demand_for_Los_Angeles_Department_of_Water_and_Power_(LDWP)_Hourly.csv',
                   header=4)
data1 = pd.read_csv('Day-ahead_demand_forecast_for_Los_Angeles_Department_of_Water_and_Power_(LDWP)_Hourly.csv',
                   header=4).iloc[16:,:]
total_data = data.merge(data1,right_on='Category',left_on='Category',how='left',suffixes=('left','right'))

#rename columns
total_data.columns = ['Date','Demand(MW)','Forecast(MW)']

#write initial data to file(first time only)
total_data.to_csv('Capstone2data.csv')

In [3]:
data = pd.read_csv('Capstone2data.csv').iloc[:,1:]
data.info()
data['Date'] = data['Date'].astype(datetime)
data = data.set_index(pd.DatetimeIndex(data['Date'])).sort_index()
data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23792 entries, 0 to 23791
Data columns (total 3 columns):
Date            23792 non-null object
Demand(MW)      23766 non-null float64
Forecast(MW)    23678 non-null float64
dtypes: float64(2), object(1)
memory usage: 557.7+ KB


,Date,Demand(MW),Forecast(MW)
Date,,,
2015-07-01 00:00:00,07/1/2015 00H,5250.0,5587.0
2015-07-01 01:00:00,07/1/2015 01H,4847.0,5447.0
2015-07-01 02:00:00,07/1/2015 02H,4593.0,5220.0
2015-07-01 03:00:00,07/1/2015 03H,4473.0,4960.0
2015-07-01 04:00:00,07/1/2015 04H,4512.0,4727.0


In [4]:
#Calculate total time span of data
time_lapse = data.index[-1] - data.index[1]
time_span = []
for i in range(time_lapse.days*24 + 16):
    time_span.append(data.index[0] + timedelta(hours=i))

In [5]:
#grouped by day df
day1 = data.groupby(pd.Grouper(freq='1D')).apply(np.sum).iloc[:,1:]
day1.describe()

#create day of the week feature
l=[]
for i in day1.index:
    d = i.isoweekday()
    l.append(d)
day1['DofWk'] = l
weekdays = pd.DataFrame(day1['Demand(MW)'].values,index=l,columns=['Demand(MW)'])
weekdays.index=weekdays.index.map(str)

#create holiday feature
holidayapi = dict(holidays.US(state='CA', years=[2015,2016,2017]))
holiday = pd.DataFrame.from_dict(holidayapi,orient='index')  
day1 = day1.merge(holiday,left_index=True,right_index=True,how='left'
                  ,suffixes=('left','right'))
l=[]
for i in day1[0].isnull():
    if i==True:
        l.append(0)
    else:
        l.append(1)
day1['holiday'] = l
day1[0] = day1[0].fillna('none')
day1.columns = ['Demand(MW)', 'Forecast(MW)', 'DofWk', 'Name', 'Holiday']
day1.head()

,Demand(MW),Forecast(MW),DofWk,Name,Holiday
Date,,,,,
2015-07-01,95002.0,101780.0,3,none,0
2015-07-02,91803.0,100735.0,4,none,0
2015-07-03,87131.0,95400.0,5,Independence Day (Observed),1
2015-07-04,78667.0,84340.0,6,Independence Day,1
2015-07-05,70900.0,76033.0,7,none,0


In [2]:
#-------------------------------------------------WEBSCRAPPING
#---------------create weather feature      
d = {}
for i in range(len(day1)):
    try:
        r1=requests.get("https://www.wunderground.com/history/airport/KCQT/%s/%s/%s/DailyHistory.html?HideSpecis=1"%(day1.index[i].year,day1.index[i].month,day1.index[i].day))
        data1=r1.content
        soup1=BeautifulSoup(data1,"html.parser")
        finder = soup1.find_all("table",{"class":"obs-table responsive"})
        finder = finder[0].find_all("tr",{'class':'no-metars'})
        l = []
        for j in finder:
            time = j.find_all("td")
            if '7' in str(time[0].text):
                try:
                    l.append(time[1].find("span",{'class':'wx-value'}).text)
                except:
                    l.append(0.0)
        d[datetime(day1.index[i].year,day1.index[i].month,day1.index[i].day)] = l
        print(i)
    except:
        print('no data for',i)

#save scraped weather data
weather = pd.DataFrame.from_dict(d,orient='index') 
weather = weather.fillna(0)
weather.to_csv('weather_data.csv')

In [7]:
#add weather feature to data grouped by day
wdata = pd.read_csv('weather_data.csv',index_col = 0,parse_dates=True)
day1 = day1.merge(wdata,left_index=True,right_index=True,
                     how='outer',suffixes=('left','right')).fillna(0)

#process weather data for original hourly data
weather = []
for i in range(1007):
    if i != 998 or 999:
        w = day1.iloc[i,5:].tolist()
        weather.append(w)
    elif i == 998 or 999:
        weather.append([0]*24)
        print(i)    
weather = [element for list_ in weather for element in list_]
#load weather data using time span as index
load_weather = pd.DataFrame(weather[:24160],index=time_span,columns=['Temp'])

In [8]:
#-------------create daylight feature
light = []
a = 1
a1=0
for i in range(len(day1)):
    try:
        r1=requests.get("https://www.wunderground.com/history/airport/KCQT/%s/%s/%s/DailyHistory.html?HideSpecis=1"%(day1.index[i].year,day1.index[i].month,day1.index[i].day))
        data1=r1.content
        soup1=BeautifulSoup(data1,"html.parser")
        finder = soup1.find_all("div",{'class':'wx-module simple'})[2]
        finder = finder.find_all("tr")[1]
        finder = finder.find_all("td")
        amhour = day1.index[i].replace(hour=int(finder[1].text[0]))
        pmhour = day1.index[i].replace(hour=int(finder[2].text[0])+12)
        l = []
        for j in range(0+24*a1,24*a):
            time = load_weather.index[j]
            if amhour < time < pmhour:
                light.append(1)
            else:
                light.append(0)
        a+=1
        a1+=1
    except:
        print('no data for',i)


no data for 1006


In [10]:
#------------------------------------------Loading data in Original
#Merge original data with weather and holiday data
growth1 = data.merge(load_weather,left_index=True,right_index=True,how='outer')
growth2 = growth1.merge(holiday,left_index=True,
                        right_index=True,how='left').fillna(method='ffill',limit=23)
#Encode hoilday variable
l=[]
for i in growth2[0].isnull():
    if i==True:
        l.append(0)
    else:
        l.append(1)
growth2['holiday'] = l

#load in remaining data
load_dow = pd.DataFrame(day1['DofWk'])
growth3 = growth2.merge(load_dow,left_index=True,
                        right_index=True,how='left').fillna(method='ffill',limit=23)
growth3.columns = ['Date', 'Demand(MW)', 'Forecast(MW)', 'Temp', 'Name', 'holiday', 'DofWk']
growth3['daylight'] = light
growth3.head()


,Date,Demand(MW),Forecast(MW),Temp,Name,holiday,DofWk,daylight
2015-07-01 00:00:00,07/1/2015 00H,5250.0,5587.0,68.0,NaN,0,3.0,0
2015-07-01 01:00:00,07/1/2015 01H,4847.0,5447.0,68.0,NaN,0,3.0,0
2015-07-01 02:00:00,07/1/2015 02H,4593.0,5220.0,66.9,NaN,0,3.0,0
2015-07-01 03:00:00,07/1/2015 03H,4473.0,4960.0,70.0,NaN,0,3.0,0
2015-07-01 04:00:00,07/1/2015 04H,4512.0,4727.0,69.1,NaN,0,3.0,0
